
# Estimate calibration factor alpha from 4D beads data (xyzt)

This file contains a basic pipeline for QP retrieval from brightfield stacks
* 4D image stack loading
* 4D stack preprocessing
* processing parameters definition
* phase calculation
* display of the results


This program is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License, with this program.  If not, see <http://www.gnu.org/licenses/>.


# load modules

In [7]:
import numpy as np
import os
#import napari
#from utils.io import loadData, writeData, loadDataOld
from utils.cropXY import cropXY
from utils.cropCoregMask import cropCoregMask
from utils.phase_structure import phase_structure
from utils.getQP import getQP
#from utils.plotStack import plotStack

In [2]:
# modify QP retrieval to iterate gamma output
import numpy as np
from numpy import dot, pi, sin, cos, angle, logical_and, logical_or
import numpy.fft as fft
import math
import time
from utils.getMirroredStack import getMirroredStack
from utils.map3D import map3D
from skimage import io

def getQP(stack=None,struct=None,mask=None):

    # mirror the data and compute adequate Fourier space grid
    kx,kz,stackM = getMirroredStack(stack,struct)
    
    if mask is None: # if no mask is provided
        # compute usefull stuff
        th=math.asin(struct.optics_NA / struct.optics_n)
        th_ill=math.asin(struct.optics_NA_ill / struct.optics_n)
        k0max = dot(dot(struct.optics_n,2),pi) / (struct.optics_wv - struct.optics_dlambda / 2) 
        k0min = dot(dot(struct.optics_n,2),pi) / (struct.optics_wv + struct.optics_dlambda / 2) 
        
        # compute Fourier space grid and the phase mask
        Kx,Kz=np.meshgrid(kx,kz)
        if struct.optics_kzT is None:
            mask2D = Kz >= np.dot(k0max,(1 - cos(th_ill)))
        else:
            mask2D = Kz >= struct.optics_kzT


        if struct.proc_applyFourierMask:  #  => compute the CTF mask for extra denoising
            # CTF theory
            maskCTF = logical_and(logical_and(logical_and(
                ((Kx - dot(k0max,sin(th_ill))) ** 2 + (Kz - dot(k0max,cos(th_ill))) ** 2) <= k0max ** 2, \
                    ((Kx + dot(k0min,sin(th_ill))) ** 2 + (Kz - k0min) ** 2) >= k0min ** 2), Kx >= 0), \
                    Kz < dot(k0max,(1 - cos(th))))            
            maskCTF = logical_or(maskCTF,maskCTF[:,::-1])
            mask2D = np.asanyarray(logical_and(mask2D,maskCTF), dtype=int)
        # since we assume a circular symetric CTF, we expand the 2Dmask in 3D
        mask=map3D(mask2D)
        #np.save("data\mask3D.npy", mask)

    # Cross-Spectral Density calculation
    print("Shifting FFT..")
    Ik=fft.fftshift(fft.fftn(fft.fftshift(stackM)))
    Gamma=np.multiply(Ik,mask)          # cross-spectral density
    print("Inverse FFT..")
    csd=fft.ifftshift(fft.ifftn(fft.ifftshift(Gamma)))
    csd = csd[:stack.shape[0], :stack.shape[1], :stack.shape[2]]   # remove the mirrored input
    
    QP = angle(csd + np.mean(np.ravel(stack)) / struct.optics_alpha)

    return QP,mask,Gamma

# load data

In [3]:
#%% 3D image stack loading
file = r"C:\Users\mengelhardt\data\local\20220922\phase_calibration\1um_FOV1\stack_1\cropped\bead1_crop.tif"
#z = 4
#t = 50
#stack, fname = loadData(file,"tif", z,t) # loadData also takes filepath as input
stack = io.imread(file)
# flag if you want to explore the data in napari
PLOT_FLAG = True 
print(stack.shape)

(251, 109, 112)


## assign dimensions accordingly
permute if necessary (order should be x,y,z,t) 

In [5]:
Nz,Ny,Nx = stack.shape
print(stack.shape)

(251, 109, 112)


# 3D stack Preprocess stack

In [6]:
if Nz == 8:                                 # i.e. multiplane data: remove the coregistration mask
    stack=cropCoregMask(stack)
    TEMP_NPIX = 100                         #  variable was undefined in mat file, set accordingly         
    stack=cropXY(stack,TEMP_NPIX - 4)       #  extra safety crop 
else:
    stack=cropXY(stack)
Nx,Ny,Nz,Nt = stack.shape
print(stack.shape)

NameError: name 'cropXY' is not defined

## permute input stack if necessary

In [8]:
NEW_ORDER = [1,2,0]
stack = np.transpose(stack, NEW_ORDER)

Nx,Ny,Nz = stack.shape
print(stack.shape)

(109, 112, 251)


In [9]:
# Phase retrieval
# define optics and processing parameters
s=phase_structure()

s.optics_kzT = 0.01                 # Axial cutoff frequency
#s.optics_kzT = 1.3 * (np.pi/0.455)*(1-np.sqrt(1-(0.58)**2))
# if set to [], use the theoretical value
s.proc_mirrorX = False              # mirror the input stack along X 
s.proc_mirrorZ = True               # mirror the input stack along Z
s.proc_applyFourierMask = True
# set experimental parameters
if Nz == 4:              # i.e. MultiPlane data
    s.optics_dz = 0.72               # [um]
else:
    s.optics_dz = 0.2               # typical sampling for fixed cells
    
phase_structure.summarise(s)

Phase structure: 
_________________
s.optics_dx = 0.1083 	 	 s.optics_wv = 0.58
s.optics_dz = 0.2 	 	 s.optics_dlambda = 0.075
s.optics_NA = 1.3 	 	 s.optics_alpha = 3.9
s.optics_NA_ill = 0.55 	 	 s.optics_kzT = 0.01
s.optics_n = 1.406
Processing paramters: 
_________________
s.proc_mirrorX = False 	 	 s.proc_mirrorZ = True
s.proc_applyFourierMask = True


# compute the phase

In [103]:
QP = np.empty(stack.shape)
Gamma = np.empty((stack.shape[0], stack.shape[1], 2*stack.shape[2], stack.shape[3]), dtype=complex)

for timepoint in range(Nt):
    print(f"Processing timepoint {timepoint}")
    QP[:,:,:,timepoint], mask, Gamma[:,:,:,timepoint] = getQP(stack[:,:,:,timepoint],s)

Processing timepoint 0
Shifting FFT..
Inverse FFT..
Processing timepoint 1
Shifting FFT..
Inverse FFT..
Processing timepoint 2
Shifting FFT..
Inverse FFT..
Processing timepoint 3
Shifting FFT..
Inverse FFT..
Processing timepoint 4
Shifting FFT..
Inverse FFT..
Processing timepoint 5
Shifting FFT..
Inverse FFT..
Processing timepoint 6
Shifting FFT..
Inverse FFT..
Processing timepoint 7
Shifting FFT..
Inverse FFT..
Processing timepoint 8
Shifting FFT..
Inverse FFT..
Processing timepoint 9
Shifting FFT..
Inverse FFT..
Processing timepoint 10
Shifting FFT..
Inverse FFT..
Processing timepoint 11
Shifting FFT..
Inverse FFT..
Processing timepoint 12
Shifting FFT..
Inverse FFT..
Processing timepoint 13
Shifting FFT..
Inverse FFT..
Processing timepoint 14
Shifting FFT..
Inverse FFT..
Processing timepoint 15
Shifting FFT..
Inverse FFT..
Processing timepoint 16
Shifting FFT..
Inverse FFT..
Processing timepoint 17
Shifting FFT..
Inverse FFT..
Processing timepoint 18
Shifting FFT..
Inverse FFT..
Pro

In [74]:
re = np.real(Gamma)
im = np.imag(Gamma)

In [99]:
print(f"coords: {np.argmax(QP[:,:,1,16].squeeze(), axis= 1)}")
print(f"coords: {np.argmax(QP[:,:,1,16].squeeze(), axis=0)}")
m = 0
for i in range(QP.shape[3]):
    if m < np.max(QP[:,:,1,i]):
        m = np.max(QP[:,:,1,i])
    print(f"{i}: {np.max(QP[:,:,1,i])}")
print(f"max Re: {np.max(re)}")
print(f"max Im: {np.max(re)}")

coords: [  4   4   4   1   1  64  67   7   6   6   2   2  33  69  45  46  46  47
  58  44  44  26  84  46  67  67  83  83  64  77  77  63  63  68  68  98
  48  53  44  44  31  31  56  56  55  31  27  27  28   1  25  26  26  47
  47  47  48  48  48  48  47  46  25  25  25  28  28  28  56  32  31  31
  52  33  33  48  68  44   4  90  73  73  82  88  89   6   6  11  16  20
   6   5  63  97  90  58  35   8  15  75  26  33  33  41  96  96 103  25]
coords: [  4   3   1   1   1   1   8   7   7   3   3  87  87  58  58  98  69   8
   8  90  17  17  55  56  56  63  58  46  65  44  71  40  73  73  57  71
  72  15  73  75  76  56  56  56  56  56  57  57  57  57  72  72  73  72
  16  44  43  68   3  31   8  54  32  32   5   6   6   7   7  13  67  62
  31  81  65  99  29  29  28  26  91  45  45  27  22  35  18  18  83  84
  79  13  11   5   5 104 104 104  93  68  28  32  24 106 107  15  15 103]
0: 0.6531237326411733
1: 0.6330623951663177
2: 0.6222720985351914
3: 0.6099012695311719
4: 0.6841713867472

In [106]:
idx = np.where(QP == m) 
idx
Gamma

array([[[[ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j, -0.+0.j, -0.+0.j, ...,  0.+0.j, -0.+0.j, -0.+0.j],
         [-0.+0.j,  0.+0.j, -0.+0.j, ..., -0.+0.j,  0.+0.j,  0.+0.j],
         ...,
         [ 0.+0.j,  0.-0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.-0.j],
         [ 0.-0.j,  0.+0.j,  0.-0.j, ...,  0.-0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  0.-0.j,  0.-0.j, ...,  0.+0.j,  0.-0.j,  0.-0.j]],

        [[ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j, -0.+0.j, -0.+0.j, ..., -0.+0.j,  0.-0.j,  0.+0.j],
         [ 0.-0.j,  0.+0.j,  0.+0.j, ..., -0.+0.j,  0.+0.j,  0.-0.j],
         ...,
         [ 0.+0.j,  0.-0.j,  0.-0.j, ...,  0.+0.j,  0.+0.j, -0.+0.j],
         [ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.-0.j, -0.+0.j,  0.+0.j],
         [-0.+0.j,  0.+0.j,  0.-0.j, ...,  0.-0.j,  0.+0.j,  0.-0.j]],

        [[ 0.+0.j,  0.+0.j,  0.+0.j, ...,  0.+0.j,  0.+0.j,  0.+0.j],
         [ 0.+0.j,  0.-0.j,  0.+0.j, ..., -0.+0.j,  0.+0.j

# display phase and input stack next to each other

In [104]:
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import *

def update(z=0, t = 0):
    fig = plt.figure(figsize=(20, 20))
    plt.subplot(121)
    plt.imshow(stack[:, :, z, t], cmap='gray')
    plt.colorbar(label="[e-/ADU]", orientation="vertical", fraction=0.046, pad=0.04)
    plt.title("input stack")
    
    plt.subplot(122)
    plt.imshow(QP[:, :, z, t], cmap='gray')
    plt.title("QP")
    plt.colorbar(label="[rad]", orientation="vertical", fraction=0.046, pad=0.04)

    plt.tight_layout()

    fig.canvas.flush_events()
    

interact(update, z= widgets.IntSlider(value=1, min=0, max=stack.shape[2]-1, step=1, description="Select Z", continuous_update=True),
                 t = widgets.IntSlider(value=1, min=0, max=stack.shape[3]-1, step=1, description="Select T", continuous_update=True))

interactive(children=(IntSlider(value=1, description='Select Z', max=3), IntSlider(value=1, description='Selec…

<function __main__.update(z=0, t=0)>

In [76]:
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import *

def update(z=0, t = 0):
    fig = plt.figure(figsize=(20, 20))
    plt.subplot(121)
    plt.imshow(stack[:, :, z, t], cmap='gray')
    plt.colorbar(label="[e-/ADU]", orientation="vertical", fraction=0.046, pad=0.04)
    plt.title("input stack")
    
    plt.subplot(122)
    plt.imshow(im[:, :, z, t], cmap='gray')
    plt.title("Im")
    plt.colorbar(label="[rad]", orientation="vertical", fraction=0.046, pad=0.04)

    plt.tight_layout()

    fig.canvas.flush_events()
    

interact(update, z= widgets.IntSlider(value=1, min=0, max=stack.shape[2]-1, step=1, description="Select Z", continuous_update=True),
                 t = widgets.IntSlider(value=1, min=0, max=stack.shape[3]-1, step=1, description="Select T", continuous_update=True))

interactive(children=(IntSlider(value=1, description='Select Z', max=3), IntSlider(value=1, description='Selec…

<function __main__.update(z=0, t=0)>

In [68]:
def updateXZ(x = 0, z = 0):
    fig = plt.figure(figsize=(10, 10))
    plt.subplot(121)
    plt.imshow(stack[x, :, z, :], cmap='gray')
    plt.colorbar(label="[e-/ADU]", orientation="vertical", fraction=0.046, pad=0.04)
    plt.title("input stack")
    
    plt.subplot(122)
    plt.imshow(QP[x, :, z, :], cmap='gray')
    plt.title("QP")
    plt.colorbar(label="[rad]", orientation="vertical", fraction=0.046, pad=0.04)
    plt.tight_layout()

    fig.canvas.flush_events()
    

interact(updateXZ, z= widgets.IntSlider(value=1, min=0, max=stack.shape[2]-1, step=1, description="Select Z", continuous_update=True),
                 x = widgets.IntSlider(value=1, min=0, max=stack.shape[0]-1, step=1, description="Select X", continuous_update=True))

interactive(children=(IntSlider(value=1, description='Select X', max=332), IntSlider(value=1, description='Sel…

<function __main__.updateXZ(x=0, z=0)>

# write QP to same folder as input with "QP_" name qualifier 

In [19]:
separator = "/"
parse = fname.split(separator)
output_path = separator.join(parse[:-1])+separator
file = parse[-1]
writeData(QP, output_path, file)

4d
(50, 333, 333, 4)
writing QP stack C:/Users/mengelhardt/data/local/20220815/2micron_beads_FOV_5/registered/QP_registered_1660642847.72465_centercrop.tif finished.


In [25]:
10.8*0.26*0.1

0.28080000000000005

In [23]:
10.8*0.26*1

2.8080000000000003

In [24]:
10.8*0.26*2

5.6160000000000005